In [ ]:
# CELL - 1
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 KB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 6.8 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubun

In [ ]:
# CELL - 2
from selenium import webdriver
import pandas as pd
from google.colab import files
from selenium.webdriver.common.by import By
from urllib.parse import unquote
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.remote.webelement import WebElement
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

<ipython-input-2-0fc948d512f3>:13: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)


In [ ]:
url = "http://talkingdictionary.swarthmore.edu/santali/?initial=a&page=1"

In [ ]:
driver.get(url)
next_page =  driver.find_element(by=By.CLASS_NAME, value='next_page')
prev_page = driver.find_element(by=By.CLASS_NAME,value="prev_page")
all_pages = driver.find_elements(by=By.CLASS_NAME,value="page")

In [ ]:
all_pages.append(next_page)
all_pages.append(prev_page)
print(len(all_pages))

27


In [ ]:
def getLinks(el):
  link = unquote(el.get_attribute("href"))
  return link

In [ ]:
links = list(map(getLinks,all_pages))

# Begin Scrapping

In [ ]:
def extract_data_from_el(el:WebElement):
  try:
    link,name,pos_type = getLinks(el.find_element(by=By.CLASS_NAME,value="audio-file")),el.find_element(by=By.CLASS_NAME,value="gloss").text,el.find_element(by=By.CLASS_NAME,value="pos").text
    return [link,name,pos_type]
  except NoSuchElementException:
    return []

In [ ]:
def get_page_detail(link):
  driver.get(link)
  data_divs = driver.find_elements(by=By.CLASS_NAME, value="insides")
  raw_data = list(map(extract_data_from_el,data_divs))
  filter_data = []
  for x in raw_data:
    if len(x) > 0:
      filter_data.append(x)
  return filter_data

In [ ]:
def start_scrapping(links):
  audio,english,pos = [],[],[]
  for link in links:
    print("scrapping",link)
    data = get_page_detail(link)
    for entry in data:
      audio.append(entry[0])
      english.append(entry[1])
      pos.append(entry[2])
  return [audio,english,pos]

In [ ]:
[audio,english,pos] = start_scrapping(links)

scrapping http://talkingdictionary.swarthmore.edu/santali/?initial=b&page=1
scrapping http://talkingdictionary.swarthmore.edu/santali/?initial=c&page=1
scrapping http://talkingdictionary.swarthmore.edu/santali/?initial=d&page=1
scrapping http://talkingdictionary.swarthmore.edu/santali/?initial=ʤ&page=1
scrapping http://talkingdictionary.swarthmore.edu/santali/?initial=ɖ&page=1
scrapping http://talkingdictionary.swarthmore.edu/santali/?initial=e&page=1
scrapping http://talkingdictionary.swarthmore.edu/santali/?initial=ə&page=1
scrapping http://talkingdictionary.swarthmore.edu/santali/?initial=ɛ&page=1
scrapping http://talkingdictionary.swarthmore.edu/santali/?initial=g&page=1
scrapping http://talkingdictionary.swarthmore.edu/santali/?initial=h&page=1
scrapping http://talkingdictionary.swarthmore.edu/santali/?initial=i&page=1
scrapping http://talkingdictionary.swarthmore.edu/santali/?initial=j&page=1
scrapping http://talkingdictionary.swarthmore.edu/santali/?initial=k&page=1
scrapping ht

In [3]:
df = pd.DataFrame(list(zip(audio,english,pos)),columns=["name","english","pos"])
df.to_csv("data.csv",index=False)
# files.download("data.csv");

NameError: ignored

# Download Files

In [5]:
from google.colab import drive
import pandas as pd
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [6]:
import requests
df = pd.read_csv("/content/gdrive/MyDrive/Santhali-English NLP/data.csv")
file_urls = df['name']
final_df = []

In [7]:
for i in range(len(file_urls)):
  new_file = requests.get(file_urls[i],stream=True)
  with open("/content/gdrive/MyDrive/Santhali-English NLP/sounds/SNT_"+str(i)+".mp3", "wb") as file: 
      for block in new_file.iter_content(chunk_size = 1024):
          if block: 
              file.write(block) 
      final_df.append(["SNT_"+str(i)+".mp3",df["english"][i],df["pos"][i]])

In [9]:
final_df

[['SNT_0.mp3', 'brother-in-law', 'noun'],
 ['SNT_1.mp3', 'father', 'noun'],
 ['SNT_2.mp3', 'itch', 'noun'],
 ['SNT_3.mp3', 'flower garden', 'noun'],
 ['SNT_4.mp3', 'outside', 'adposition'],
 ['SNT_5.mp3', 'enemy', 'noun'],
 ['SNT_6.mp3', 'baby', 'noun'],
 ['SNT_7.mp3', 'baby', 'noun'],
 ['SNT_8.mp3', 'bear', 'noun'],
 ['SNT_9.mp3', 'both', 'adjective'],
 ['SNT_10.mp3', 'no', 'adverb'],
 ['SNT_11.mp3', 'two', 'noun'],
 ['SNT_12.mp3', 'twenty', 'noun'],
 ['SNT_13.mp3', 'toad', 'noun'],
 ['SNT_14.mp3', 'vegetable garden', 'noun'],
 ['SNT_15.mp3', 'raw', 'adjective'],
 ['SNT_16.mp3', 'sadly', 'adverb'],
 ['SNT_17.mp3', 'bat', 'noun'],
 ['SNT_18.mp3', 'slowly', 'adverb'],
 ['SNT_19.mp3', 'slow', 'adjective'],
 ['SNT_20.mp3', 'white ant hill', 'noun'],
 ['SNT_21.mp3', 'bad', 'adjective'],
 ['SNT_22.mp3', 'ghost', 'noun'],
 ['SNT_23.mp3', 'egg', 'noun'],
 ['SNT_24.mp3', 'ripe', 'adjective'],
 ['SNT_25.mp3', 'snake', 'noun'],
 ['SNT_26.mp3', 'spider', 'noun'],
 ['SNT_27.mp3', 'boar', 'noun'],
